<p style = "font-size : 42px; color : #393e46 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #00adb5; border-radius: 5px 5px;"><strong>Simple 2-Layerd(1-hidden) Neural Network with GD & SGD</strong></p>

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('train.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [5]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T # transpose to get the data in a row-wise format
Y_dev = data_dev[0] # first row is the label
X_dev = data_dev[1:n] # rest of the rows are the pixel values
X_dev = X_dev / 255. # normalize pixel values in range 0-1

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape # m_train is the number of training examples and _ is the number of features

In [6]:
Y_train

array([1, 1, 4, ..., 9, 8, 1], dtype=int64)

In [7]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

<p style = "font-size : 42px; color : #393e46 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #00adb5; border-radius: 5px 5px;"><strong>Gradient Descent</strong></p>

In [ ]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

In [ ]:
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [9]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 501)

Iteration:  0
[1 3 3 ... 3 8 8] [1 1 4 ... 9 8 1]
0.12068292682926829
Iteration:  10
[1 3 3 ... 5 8 1] [1 1 4 ... 9 8 1]
0.22234146341463415
Iteration:  20
[1 3 0 ... 0 8 1] [1 1 4 ... 9 8 1]
0.27970731707317076
Iteration:  30
[1 8 4 ... 4 8 1] [1 1 4 ... 9 8 1]
0.33982926829268295
Iteration:  40
[1 8 4 ... 9 8 1] [1 1 4 ... 9 8 1]
0.40351219512195124
Iteration:  50
[1 8 4 ... 9 8 1] [1 1 4 ... 9 8 1]
0.4676585365853659
Iteration:  60
[1 8 4 ... 9 8 1] [1 1 4 ... 9 8 1]
0.5180487804878049
Iteration:  70
[1 8 4 ... 9 8 1] [1 1 4 ... 9 8 1]
0.5574878048780488
Iteration:  80
[1 8 4 ... 9 8 1] [1 1 4 ... 9 8 1]
0.5892682926829268
Iteration:  90
[1 8 4 ... 9 8 1] [1 1 4 ... 9 8 1]
0.6139512195121951
Iteration:  100
[1 8 4 ... 9 8 1] [1 1 4 ... 9 8 1]
0.6358048780487805
Iteration:  110
[1 8 4 ... 9 8 1] [1 1 4 ... 9 8 1]
0.6541951219512195
Iteration:  120
[1 8 4 ... 9 8 1] [1 1 4 ... 9 8 1]
0.6709268292682927
Iteration:  130
[1 8 4 ... 9 8 1] [1 1 4 ... 9 8 1]
0.6865365853658537
Iteration:  

In [8]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

In [10]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[3 7 8 6 5 3 1 5 3 1 0 7 7 6 2 4 0 9 0 7 0 7 3 6 5 5 6 5 8 7 7 9 4 7 3 1 7
 2 1 8 5 0 3 8 3 6 4 8 6 0 2 8 1 5 6 9 0 3 6 0 6 6 9 1 6 7 8 8 0 3 0 8 4 4
 2 5 6 8 9 9 8 4 1 1 0 6 3 5 4 3 9 4 6 8 2 7 4 2 7 1 3 6 4 8 0 8 2 6 4 4 1
 2 1 1 1 2 7 1 1 2 5 9 6 9 6 8 1 7 8 3 7 8 8 4 1 1 7 6 2 6 9 7 6 0 8 5 2 4
 9 5 3 1 0 6 4 0 5 8 6 3 5 9 3 3 3 0 2 8 7 8 2 9 8 3 4 8 5 6 1 5 5 1 2 7 3
 8 1 7 0 9 5 5 9 2 9 7 2 3 2 8 4 7 9 6 2 3 4 7 1 3 6 1 4 5 7 0 9 1 4 3 0 3
 3 2 6 3 6 7 4 1 4 3 7 3 7 3 9 4 0 9 9 3 6 1 6 1 6 9 4 3 8 1 8 5 2 7 2 3 2
 1 8 0 0 5 6 0 4 4 4 8 1 3 0 7 3 0 1 8 3 3 8 8 1 5 3 9 4 5 6 0 6 3 4 7 3 4
 2 7 1 9 4 0 1 9 1 7 4 9 9 5 8 6 6 0 0 9 2 4 8 5 8 7 0 4 8 3 0 2 9 7 0 6 8
 0 6 8 3 1 2 1 6 6 7 1 7 9 3 1 0 4 3 0 6 7 1 8 4 9 1 7 6 4 9 8 7 1 1 0 2 3
 8 2 9 7 9 3 4 5 6 3 7 2 4 4 6 0 5 1 9 8 8 6 3 1 4 9 4 6 7 7 0 7 5 7 0 4 3
 8 9 3 8 1 6 2 0 3 9 4 3 2 7 2 5 4 5 3 5 8 1 5 4 8 9 8 7 7 5 9 4 8 6 9 1 8
 7 0 8 1 5 3 6 9 6 8 1 4 1 8 9 8 4 4 1 0 0 7 4 8 3 6 3 1 5 2 0 6 8 2 2 7 1
 7 7 9 7 3 0 1 7 2 7 0 8 

0.846

In [11]:
# # Save the model and test it on the test set
# np.savez_compressed('simple_nn_trained_model.npz', W1=W1, b1=b1, W2=W2, b2=b2)

# test_data = pd.read_csv('test.csv')
# test_data = np.array(test_data)
# test_data = test_data.T
# test_data = test_data / 255.0
# m_test = test_data.shape[1]

# def make_test_predictions(X, W1, b1, W2, b2):
#     _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
#     predictions = get_predictions(A2)
#     return predictions

# test_predictions = make_test_predictions(test_data, W1, b1, W2, b2)

<p style = "font-size : 42px; color : #393e46 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #00adb5; border-radius: 5px 5px;"><strong>Stochastic Gradient Descent</strong></p>

In [15]:
def one_hot(Y, num_classes):
    one_hot_Y = np.zeros((num_classes, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y, 10)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

In [16]:
def stochastic_gradient_descent(X, Y, alpha, iterations, batch_size):
    W1, b1, W2, b2 = init_params()
    m = X.shape[1]  # Total number of training examples

    for i in range(iterations):
        # Shuffle the dataset and split it into mini-batches
        permutation = np.random.permutation(m)
        shuffled_X = X[:, permutation]
        shuffled_Y = Y[permutation]

        for j in range(0, m, batch_size):
            # Extract a mini-batch
            X_mini_batch = shuffled_X[:, j:j+batch_size]
            Y_mini_batch = shuffled_Y[j:j+batch_size]

            Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X_mini_batch)
            dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X_mini_batch, Y_mini_batch)
            W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y_mini_batch))
    
    return W1, b1, W2, b2

In [17]:
# Parameters for stochastic gradient descent are - X, Y, alpha, iterations, batch_size (X and Y are the training data)
W1, b1, W2, b2 = stochastic_gradient_descent(X_train, Y_train, 0.1, 501, 32)

Iteration:  0
[9 8 8 2 1 7 2 8] [5 9 4 0 3 8 1 8]
0.125
Iteration:  10
[4 2 2 1 2 1 3 1] [1 3 4 2 5 8 8 3]
0.0
Iteration:  20
[1 0 4 7 2 1 1 1] [1 7 9 0 1 1 6 1]
0.375
Iteration:  30
[2 2 9 3 9 1 8 8] [4 7 7 3 2 8 5 3]
0.125
Iteration:  40
[1 4 4 2 4 4 0 6] [1 4 5 3 5 3 8 8]
0.25
Iteration:  50
[1 4 0 4 4 0 2 2] [1 4 6 4 9 3 7 2]
0.5
Iteration:  60
[1 7 0 7 9 0 9 7] [3 7 5 3 4 0 9 7]
0.5
Iteration:  70
[7 7 7 8 9 2 7 6] [7 7 7 8 9 1 7 6]
0.875
Iteration:  80
[4 2 2 1 1 3 2 4] [6 2 8 1 6 0 2 5]
0.375
Iteration:  90
[0 6 6 1 8 2 4 6] [2 6 6 3 8 2 4 6]
0.75
Iteration:  100
[4 0 4 0 2 0 2 7] [4 0 7 5 3 0 2 4]
0.5
Iteration:  110
[7 4 3 2 0 3 9 2] [7 4 3 6 5 3 4 8]
0.5
Iteration:  120
[6 1 0 1 7 0 1 2] [6 1 0 2 8 0 1 2]
0.75
Iteration:  130
[0 0 3 4 0 7 8 4] [0 0 3 4 0 8 3 4]
0.75
Iteration:  140
[1 0 8 0 6 6 8 3] [1 0 8 0 6 6 8 2]
0.875
Iteration:  150
[0 8 5 9 8 3 1 0] [0 5 5 4 8 3 1 0]
0.75
Iteration:  160
[2 3 2 0 7 2 1 7] [2 3 2 5 7 2 1 7]
0.875
Iteration:  170
[1 4 3 9 9 9 2 9] [1 5 5

In [18]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[3 7 2 6 5 3 1 5 3 1 0 7 7 6 2 4 0 9 0 7 0 7 3 6 5 5 6 5 8 7 7 9 4 7 3 1 7
 2 1 5 5 0 3 8 3 6 4 8 6 0 2 3 1 8 6 9 0 3 6 0 6 6 7 1 6 7 8 8 0 8 0 8 4 4
 2 5 6 8 9 9 1 4 1 1 3 6 3 8 4 5 9 8 6 8 2 7 4 2 7 1 3 6 4 9 0 8 2 6 8 4 1
 2 1 1 1 2 3 1 1 2 8 9 6 9 6 8 1 9 8 3 7 8 3 4 1 1 7 6 2 6 4 7 6 0 8 5 6 4
 9 5 3 1 0 6 4 0 5 8 6 3 5 9 7 3 3 0 8 8 7 8 2 9 5 3 4 8 5 6 8 3 5 1 3 7 3
 8 1 7 0 9 5 5 9 6 9 7 3 3 2 8 4 7 9 4 2 3 4 7 1 2 6 1 4 5 7 0 9 1 4 3 0 3
 3 2 6 3 6 7 4 1 4 3 7 3 7 3 4 4 0 9 9 3 6 3 6 1 6 9 4 3 1 1 8 5 2 7 2 3 7
 1 8 0 0 3 6 0 4 4 4 8 1 3 0 7 3 0 1 8 3 3 9 8 1 5 3 9 4 5 6 0 6 3 4 7 3 4
 2 4 1 9 4 0 1 9 1 7 4 9 9 3 8 6 6 0 0 9 2 9 8 5 1 7 0 4 8 3 0 3 9 7 6 6 8
 0 6 4 3 1 2 1 6 6 7 1 9 9 3 1 0 4 3 0 6 7 1 8 4 9 1 7 6 4 9 8 9 1 1 0 2 3
 8 2 9 7 9 3 4 8 6 3 7 2 4 4 6 0 5 2 9 8 8 6 3 1 4 7 4 6 3 7 0 7 7 7 0 4 3
 8 9 3 8 1 6 2 0 3 9 9 3 0 7 2 5 9 5 3 5 8 1 5 4 8 9 8 7 7 5 9 4 8 6 9 1 8
 7 0 6 1 5 3 6 9 4 8 1 4 1 8 9 8 4 9 1 0 0 7 4 8 3 6 3 8 5 9 0 2 8 2 2 7 1
 7 7 9 7 3 0 1 7 2 1 0 8 

0.855

In [23]:
# Now this time lets try again with different parameters

W1, b1, W2, b2 = stochastic_gradient_descent(X_train, Y_train, 0.2, 501, 16)

Iteration:  0
[8 7 7 0 0 7 2 5] [8 1 6 6 9 0 6 3]
0.125
Iteration:  10
[1 6 0 0 5 2 7 7] [5 2 7 9 9 2 7 6]
0.25
Iteration:  20
[4 7 7 4 6 0 2 8] [9 9 3 9 6 7 6 9]
0.125
Iteration:  30
[5 9 3 6 9 7 0 1] [1 7 1 2 9 7 4 8]
0.25
Iteration:  40
[1 3 0 3 3 1 0 9] [8 2 0 3 8 3 4 4]
0.25
Iteration:  50
[4 5 1 4 6 9 0 6] [7 5 1 8 0 9 0 6]
0.625
Iteration:  60
[1 9 7 3 2 4 4 3] [1 9 7 3 5 4 7 1]
0.625
Iteration:  70
[7 7 3 6 9 4 2 7] [7 7 3 6 9 4 2 7]
1.0
Iteration:  80
[1 8 3 4 8 2 0 9] [2 3 3 4 2 2 3 7]
0.375
Iteration:  90
[6 6 2 6 3 0 0 3] [6 5 6 6 3 0 9 3]
0.625
Iteration:  100
[3 5 4 3 4 3 7 9] [3 5 4 3 3 3 8 9]
0.75
Iteration:  110
[9 5 7 8 3 1 2 1] [9 3 7 8 3 8 6 1]
0.625
Iteration:  120
[5 7 4 8 7 2 3 8] [5 7 9 8 7 6 3 8]
0.75
Iteration:  130
[3 2 7 6 0 5 5 2] [1 2 7 6 0 5 5 2]
0.875
Iteration:  140
[3 7 6 6 5 2 2 5] [3 7 6 2 5 2 2 5]
0.875
Iteration:  150
[0 4 4 0 4 2 1 4] [0 4 4 0 9 2 1 4]
0.875
Iteration:  160
[1 3 2 2 8 5 2 5] [1 3 2 7 3 5 2 5]
0.75
Iteration:  170
[8 3 3 9 3 8 7 9]

In [24]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[3 7 2 6 5 3 1 5 3 1 0 7 7 6 2 4 0 9 0 7 0 7 3 6 0 5 6 5 8 7 7 9 9 7 3 1 7
 2 1 5 5 0 3 8 3 6 4 8 6 0 2 8 1 5 6 9 0 3 6 0 6 6 7 1 6 7 7 8 0 8 0 8 4 5
 2 5 6 8 9 9 8 2 7 1 3 4 3 8 4 5 9 4 6 8 2 7 4 2 7 1 3 6 4 8 0 8 2 6 5 4 1
 2 1 1 1 2 7 1 1 2 8 9 6 9 6 8 1 9 8 3 7 5 3 4 1 1 7 6 2 6 9 7 6 0 8 0 2 4
 9 5 3 1 0 6 4 0 5 8 6 3 8 9 9 3 3 0 8 8 7 8 3 9 5 3 4 8 5 6 8 8 5 1 2 7 3
 8 1 7 0 9 5 5 9 4 9 7 2 3 2 8 4 7 9 4 2 3 4 7 1 2 6 1 4 5 9 0 9 1 4 3 0 3
 3 2 6 3 4 7 4 1 4 3 7 3 7 5 9 4 5 9 9 3 6 2 6 1 6 9 4 3 9 1 8 5 2 7 2 3 7
 1 1 0 0 3 6 0 4 4 4 8 1 3 0 7 3 0 1 8 3 3 8 8 1 5 3 9 4 5 6 0 6 3 4 7 3 4
 2 7 1 9 4 0 1 9 8 7 4 4 9 5 8 6 6 0 0 9 2 7 8 5 8 7 0 4 8 3 0 2 9 7 0 6 8
 0 6 4 3 1 2 1 6 6 7 1 9 9 3 1 0 4 3 0 6 9 1 9 4 9 1 7 2 4 7 8 7 1 1 0 2 3
 8 2 9 7 3 3 4 5 6 3 7 2 4 7 6 0 5 2 9 8 8 6 3 1 4 9 6 6 7 7 0 7 5 7 0 4 3
 8 9 3 8 1 6 2 0 3 9 9 3 0 7 2 5 9 5 3 5 8 1 5 4 8 9 2 7 7 5 9 6 8 6 9 1 8
 7 0 6 1 5 3 6 9 4 8 1 4 1 8 9 8 4 4 1 0 5 7 4 8 9 6 3 8 5 2 0 6 8 2 2 7 1
 7 7 9 7 3 0 1 7 2 7 0 8 

0.888